In [17]:
from am4pa.linnea import LinneaConfig, DataManagerLinnea
from va4algs import RankingDataLinnea
from va4algs import RankingModel
from va4algs import DataAnalyzer3Way
import pandas as pd
pd.set_option('display.max_rows', None)

### Step 1) Indicate the Expression and the Experiment ID

In [2]:
EXPRESSION_FILE = '../measurements/linnea-expressions/gls.py'
EXPERIMENT_ID = 'GLS-ix01'

### Step 2) Sync Local and Back end

In [3]:
SERVER = "login18-1.hpc.itc.rwth-aachen.de"
UNAME = "as641651"
BACK_END_FOLDER = "~/PhD/va4algs-exps/"
FRONT_END_FOLDER = "../measurements/codes/"

In [4]:
linnea_config = LinneaConfig(EXPRESSION_FILE)
linnea_config.set_local_directory("{}{}".format(FRONT_END_FOLDER,EXPERIMENT_ID))
linnea_config.setup_backend_details(server=SERVER,
                                   uname = UNAME,
                                   init_script="~/.analyzer",
                                   backend_root=BACK_END_FOLDER)
linnea_config.connect_backend()

In [5]:
dml = DataManagerLinnea(linnea_config)

{'problem': 'linnea-expressions/gls.py', 'backend': True, 'server': 'login18-1.hpc.itc.rwth-aachen.de', 'uname': 'as641651', 'local_dir': 'codes/GLS-ix01', 'backend_dir': '~/PhD/va4algs-exps/GLS-ix01'}


#### Available measurements

In [6]:
print(len(dml.measurements_data['24T']))
print(len(dml.operands_data['24T']))

105
105


### Step 3) View Ranking Data

In [7]:
THREADS = '24T'
RANKING_METHOD = 3

In [8]:
rm = RankingModel(name='R{}'.format(RANKING_METHOD),method=RANKING_METHOD)
rdl = RankingDataLinnea(dml,rm,THREADS)

In [9]:
#rdl.rank3way(update=True)
rdl.load()

In [14]:
#rdl.data_anomalies

In [15]:
print("Percent Anomalies: {}".format(rdl.percent_anomalies))
print("Average Risk: {}".format(rdl.avg_risk))
print("Average Adjusted Risk: {}".format(rdl.avg_adj_risk))

Percent Anomalies: 0.14285714285714285
Average Risk: 0.08747849681183015
Average Adjusted Risk: 0.019510524887384043


In [16]:
rdl.filter_interesting_operands(adj_risk_thresh=0.0)

,op_str,num_algs,max-rel-flops,is_anomaly,n_rel0,n_anomalies,risk,time_inc,rel_inc,adj_risk,rel-flops-cutoff
60,304_247,100,0.737968,True,9,9,1.000000,0.000401,0.480986,0.480986,0.736185
62,337_249,100,0.733333,True,9,9,1.000000,0.000273,0.339285,0.339285,0.733333
51,338_206,100,0.700000,True,9,9,1.000000,0.000166,0.204306,0.204306,0.700000
45,378_172,100,0.637993,True,9,9,1.000000,0.000104,0.177785,0.177785,0.637993
74,717_82,100,0.122807,True,9,9,1.000000,0.000099,0.067785,0.067785,0.023392
57,740_50,100,0.048485,True,27,18,0.666667,0.000372,0.351967,0.234645,0.000000
31,699_47,100,0.043165,True,27,18,0.666667,0.000326,0.295212,0.196808,0.000000
56,672_35,100,0.033898,True,27,18,0.666667,0.000177,0.202571,0.135047,0.000000
73,649_17,100,0.021234,True,27,25,0.925926,0.000077,0.056578,0.052387,0.000000
86,1128_46,100,0.018484,True,27,3,0.111111,0.001053,0.427004,0.047445,0.000000


In [28]:
da = DataAnalyzer3Way(rdl.f_data_kernels, rdl.f_data_relations,rdl.f_data_ext)

/Users/aravind/.local/lib/python3.9/site-packages/va4algs/data_analyzer.py:190: RuntimeWarning: invalid value encountered in double_scalars
  df['selection score'] = df.apply(lambda x: x[1]/float(x[1]+ x[-1]), axis=1).fillna(-1).round(2)


In [29]:
da.discover_kernel_rules()

,index,kernel,case:rel-flops,-1,0,1,total,good (%),bad (%),selection score
0,3,qr,0.005848,9,0,0,9,0.0,1.0,0.0
1,4,syev,0.005848,4,0,0,4,0.0,1.0,0.0
2,5,syrk,0.000000,5,0,0,5,0.0,1.0,0.0
3,6,transpose,0.700000,3,0,0,3,0.0,1.0,0.0


In [30]:
da.discover_relations_rules()

class,index,kernelA,kernelB,-1,0,1,total,good (%),bad (%),selection score
0,6,gemm,trsv,1,0,0,1,0.0,1.0,0.0
1,7,gemv,@@endnode,8,0,0,8,0.0,1.0,0.0
2,9,gemv,gemv,9,0,0,9,0.0,1.0,0.0
3,14,gemv,trsm,4,0,0,4,0.0,1.0,0.0
4,34,trsm,potrf,4,0,0,4,0.0,1.0,0.0
5,41,trsv,gemv,5,0,0,5,0.0,1.0,0.0
6,42,trsv,potrf,5,0,0,5,0.0,1.0,0.0
7,46,trsv,trsm,10,0,0,10,0.0,1.0,0.0


In [31]:
dy = da.filter_relations_not_in_rules()
dy

class,level_0,index,kernelA,kernelB,-1,0,1,total,good (%),bad (%),selection score
0,1,2,gemm,gemv,3,2,0,5,0.0,0.60,0.0
1,3,8,gemv,gemm,1,4,0,5,0.0,0.20,0.0
2,4,10,gemv,potrf,3,2,0,5,0.0,0.60,0.0
3,5,15,gemv,trsv,5,5,0,10,0.0,0.50,0.0
4,7,17,potrf,trsm,9,5,0,14,0.0,0.64,0.0
5,8,18,potrf,trsv,11,4,0,15,0.0,0.73,0.0
6,9,32,trsm,gemm,1,4,0,5,0.0,0.20,0.0
7,10,33,trsm,gemv,6,4,0,10,0.0,0.60,0.0
8,11,37,trsm,trsm,12,4,0,16,0.0,0.75,0.0
9,12,38,trsm,trsv,9,1,0,10,0.0,0.90,0.0
